In [49]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import math
import streamlit

In [ ]:
url = "https://www.bezrealitky.cz/vyhledat?estateType=BYT&location=exact&offerType=PRONAJEM&osm_value=Ústecký+kraj%2C+Severozápad%2C+Česko&regionOsmIds=R442452&currency=CZK"

request_response = requests.get(url)
soup = BeautifulSoup(request_response.text, "html.parser")

pocet_nemovitosti_element = soup.find("div", {"class": "PropertiesMap_propertiesMapScroll__VtwqL"}) \
    .find("p", {"class": "form-label"}) \
    .find("span")
pocet_nemovitosti = pocet_nemovitosti_element.text.split("\xa0")[0]
pocet_stranek = math.ceil(int(pocet_nemovitosti) / 15)

list_adres = []
for i in range(1, pocet_stranek + 1):
    url = "https://www.bezrealitky.cz/vyhledat?estateType=BYT&location=exact&offerType=PRONAJEM&osm_value=Ústecký+kraj%2C+Severozápad%2C+Česko&regionOsmIds=R442452&currency=CZK" + f"&page={i}"

    request_response = requests.get(url)
    soup = BeautifulSoup(request_response.text, "html.parser")

    odkazy_elementy = soup.find("section", {"class": "Section_section__gjwvr"}) \
        .find_all("article")

    odkazy = []
    for element_odkaz in odkazy_elementy:
        odkaz = element_odkaz.find("a")
        odkazy.append(odkaz)

    for odkaz in odkazy:
        href = odkaz.get("href")
        if href:
            list_adres.append(href)

list_adres = list(set(list_adres))

data = []
for url in list_adres:
    request_response = requests.get(url)
    soup = BeautifulSoup(request_response.text)
    seznam_parametru = {}

    najemne = soup.find("div", {"class": "StickyBox_stickyBox__nu8sW"}) \
        .find_all("div", {"class": "ContentBox_contentBox__tD7YI"})[0] \
        .find("div", {"class": "justify-content-between"}) \
        .find_all("span")[4]
    najemne = re.sub(r'\D', '', najemne.text)
    seznam_parametru["Najemne"] = najemne

    podsekce_parametry_list = soup.find_all("div", {"class": "ParamsTable_paramsTableGroup__Flyfi"}) 

    for podsekce_parametry in podsekce_parametry_list[0:2]:
        radky_parametru = podsekce_parametry.find_all("tr")
        for radek_prametru in radky_parametru:
            popisek_parametru = radek_prametru.find("th").text
            hodnota_parametru = radek_prametru.find("td").text.replace("\xa0", " ")
            seznam_parametru[popisek_parametru] = hodnota_parametru
    
    data.append(seznam_parametru)

pd.DataFrame(data)
   

,Najemne,Dostupné od,Konstrukce budovy,Vybaveno,Číslo inzerátu,PENB,Užitná plocha,Stav,Dispozice,Podlaží,Vlastnictví,Umístění,Cena za jednotku,Stáří,Provedení,Rekonstrukce,Vytápění
0,8500,25. 10. 2025,Panel,Částečně,958784,D - Méně úsporná,31 m²,Velmi dobrý,1+kk,7. podlaží z 7,Osobní,Klidná část,274 Kč / m2,NaN,NaN,NaN,NaN
1,7000,10. 2. 2025,Cihla,Nevybaveno,798897,G - Mimořádně nehospodárná,25 m²,Velmi dobrý,1+kk,1. podlaží z 3,NaN,Klidná část,280 Kč / m2,NaN,NaN,NaN,NaN
2,10900,3. 10. 2025,Panel,Vybaveno,951890,C - Úsporná,54 m²,Dobrý,3+1,2. podlaží z 7,Osobní,Sídliště,202 Kč / m2,30 - 50 let,Standardní,NaN,NaN
3,12000,1. 10. 2025,Cihla,Částečně,772761,G - Mimořádně nehospodárná,55 m²,Velmi dobrý,2+1,2. podlaží z 2,NaN,Klidná část,218 Kč / m2,NaN,NaN,NaN,NaN
4,10000,10. 8. 2023,Cihla,Nevybaveno,818428,G - Mimořádně nehospodárná,70 m²,Dobrý,3+kk,4. podlaží z 4,NaN,Klidná část,143 Kč / m2,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,31131,26. 10. 2025,NaN,Vybaveno,830692,NaN,59 m²,NaN,NaN,5. podlaží,NaN,Klidná část,528 Kč / m2,NaN,NaN,NaN,NaN
64,7000,13. 10. 2025,Cihla,Částečně,687674,C - Úsporná,35 m²,Velmi dobrý,1+kk,NaN,Osobní,Klidná část,200 Kč / m2,NaN,Standardní,NaN,NaN
65,25078,26. 10. 2025,NaN,Vybaveno,830695,NaN,55 m²,NaN,NaN,2. podlaží,NaN,Klidná část,456 Kč / m2,NaN,NaN,NaN,NaN
66,10000,6. 10. 2025,Panel,Nevybaveno,952336,NaN,26 m²,Velmi dobrý,1+1,Přízemí,NaN,Sídliště,385 Kč / m2,NaN,NaN,NaN,NaN
